In [ ]:
model.save('final_model.h5')


In [ ]:
model.save('final_model.keras')
loaded_model = keras.models.load_model('final_model.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import os

# Check the current working directory
print("Current Directory:", os.getcwd())

# List the files in the current directory
print("Files in Directory:", os.listdir())


Current Directory: /content
Files in Directory: ['.config', 'final_model.keras', 'drive', 'final_model.h5', 'sample_data']


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the CSV files
df1 = pd.read_csv('/content/drive/MyDrive/ML_Project/malicious_dataset_2.csv')
df2 = pd.read_csv('/content/drive/MyDrive/ML_Project/safe_urls.csv')

# Display the first few rows of each DataFrame to confirm they loaded correctly
print("DataFrame 1 (Malicious Dataset):")
print(df1.head())

print("\nDataFrame 2 (Safe URLs):")
print(df2.head())


DataFrame 1 (Malicious Dataset):
  http://cnfrmappsecurhomebusiness.github.io/
0    http://f-protection-services.vercel.app/
1           https://currently4326.weebly.com/
2    https://sites.google.com/view/zxacq/home
3                        https://urlz.fr/sALl
4         https://cnnxmnwsieudjff.weebly.com/

DataFrame 2 (Safe URLs):
   1     google.com
0  2    youtube.com
1  3   facebook.com
2  4      baidu.com
3  5  wikipedia.org
4  6      yahoo.com


In [ ]:
# Add a label column to each DataFrame
df1['label'] = 0  # Malicious links
df2['label'] = 1  # Safe links

# Combine the DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Display the first few rows of the combined DataFrame
print("Combined DataFrame:")
print(combined_df.head())


Combined DataFrame:
  http://cnfrmappsecurhomebusiness.github.io/  label   1 google.com
0    http://f-protection-services.vercel.app/      0 NaN        NaN
1           https://currently4326.weebly.com/      0 NaN        NaN
2    https://sites.google.com/view/zxacq/home      0 NaN        NaN
3                        https://urlz.fr/sALl      0 NaN        NaN
4         https://cnnxmnwsieudjff.weebly.com/      0 NaN        NaN


In [ ]:
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('/content/drive/MyDrive/ML_Project/malicious_dataset_2.csv', header=None)
df2 = pd.read_csv('/content/drive/MyDrive/ML_Project/safe_urls.csv', header=0)

# Assign proper column names
df1.columns = ['url']  # Malicious URLs
df2.columns = ['url']   # Safe URLs

# Add labels to each DataFrame
df1['label'] = 0  # Malicious links
df2['label'] = 1  # Safe links

# Combine the DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Display the first few rows of the combined DataFrame
print("Combined DataFrame:")
print(combined_df.head())


ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [ ]:
# Load the safe URLs CSV and check its structure
df2 = pd.read_csv('/content/drive/MyDrive/ML_Project/safe_urls.csv', header=None)  # Load without specifying headers
print("Safe URLs DataFrame:")
print(df2.head())
print(f"Shape of df2: {df2.shape}")  # Print shape to understand the number of columns


Safe URLs DataFrame:
   0              1
0  1     google.com
1  2    youtube.com
2  3   facebook.com
3  4      baidu.com
4  5  wikipedia.org
Shape of df2: (1000000, 2)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from scipy.stats import entropy
from urllib.parse import urlparse
from google.colab import drive
from sklearn.decomposition import TruncatedSVD

# Mount Google Drive to load the dataset
drive.mount('/content/drive')

# Load dataset directly from Google Drive
data_path = '/content/drive/MyDrive/ML_Project/combined_url_dataset.xlsx'  # Update path
combined_df = pd.read_excel(data_path)

# Use a smaller sample (10%) of the data for testing
combined_df = combined_df.sample(frac=0.1, random_state=42)

# Feature Extraction for Additional Features
def calculate_entropy(url):
    """Calculate Shannon entropy of characters in the URL."""
    counts = Counter(url)
    total_chars = len(url)
    return -sum((count / total_chars) * np.log2(count / total_chars) for count in counts.values())

def subdomain_count(url):
    """Count the number of subdomains in the URL."""
    parsed_url = urlparse(url)
    return parsed_url.netloc.count('.')

# Apply feature extraction
combined_df['url_length'] = combined_df['url'].apply(len)
combined_df['entropy'] = combined_df['url'].apply(calculate_entropy)
combined_df['subdomain_count'] = combined_df['url'].apply(subdomain_count)

# Initialize the TF-IDF Vectorizer and apply dimensionality reduction
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(combined_df['url'])

# Use TruncatedSVD for dimensionality reduction
svd = TruncatedSVD(n_components=200)
X_tfidf_reduced = svd.fit_transform(X_tfidf)

# Scale and combine with additional features
X_additional = combined_df[['url_length', 'entropy', 'subdomain_count']].values
scaler = StandardScaler()
X_additional_scaled = scaler.fit_transform(X_additional)

# Combine TF-IDF reduced features with scaled additional features
X_combined = np.hstack((X_tfidf_reduced, X_additional_scaled))

# Define the target variable
y = combined_df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Build the ANN Model with a smaller structure
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9707 - loss: 0.2260 - val_accuracy: 1.0000 - val_loss: 7.2583e-04
Epoch 2/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 4.7372e-04 - val_accuracy: 1.0000 - val_loss: 1.3503e-04
Epoch 3/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.0277e-04 - val_accuracy: 1.0000 - val_loss: 4.7335e-05
Epoch 4/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 1.0000 - loss: 3.8517e-05 - val_accuracy: 1.0000 - val_loss: 2.0445e-05
Epoch 5/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 1.6348e-05 - val_accuracy: 1.0000 - val_loss: 9.7474e-06
Epoch 6/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 8.2633e-06 - val_accuracy: 1.0000 - val_loss: 4.9607e-06
Epoch 7/10
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 4.2241e-06 - val_accuracy: 1.0000 - val_loss: 2.5862e-06
Epoch 8/10
1049/1049 ━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/ML_Project/malicious_dataset_3.csv'


In [ ]:
import pandas as pd

# Load the dataset
data_path = '/content/drive/MyDrive/ML_Project/combined_url_dataset2.xlsx'
df = pd.read_excel(data_path)

# Shuffle the data
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the shuffled data back to a new Excel file
df.to_excel('/content/drive/MyDrive/ML_Project/shuffled_combined_url_dataset2.xlsx', index=False)


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from scipy.stats import entropy
from urllib.parse import urlparse
from google.colab import drive
from sklearn.decomposition import TruncatedSVD

# Mount Google Drive to load the dataset
drive.mount('/content/drive')

# Load dataset directly from Google Drive
data_path = '/content/drive/MyDrive/ML_Project/combined_url_dataset2.xlsx'  # Update path
combined_df = pd.read_excel(data_path)

# Use a smaller sample (10%) of the data for testing
combined_df = combined_df.sample(frac=0.1, random_state=42)

# Feature Extraction for Additional Features
def calculate_entropy(url):
    """Calculate Shannon entropy of characters in the URL."""
    counts = Counter(url)
    total_chars = len(url)
    return -sum((count / total_chars) * np.log2(count / total_chars) for count in counts.values())

def subdomain_count(url):
    """Count the number of subdomains in the URL."""
    parsed_url = urlparse(url)
    return parsed_url.netloc.count('.')

# Apply feature extraction
combined_df['url_length'] = combined_df['url'].apply(len)
combined_df['entropy'] = combined_df['url'].apply(calculate_entropy)
combined_df['subdomain_count'] = combined_df['url'].apply(subdomain_count)

# Initialize the TF-IDF Vectorizer and apply dimensionality reduction
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(combined_df['url'])

# Use TruncatedSVD for dimensionality reduction
svd = TruncatedSVD(n_components=200)
X_tfidf_reduced = svd.fit_transform(X_tfidf)

# Scale and combine with additional features
X_additional = combined_df[['url_length', 'entropy', 'subdomain_count']].values
scaler = StandardScaler()
X_additional_scaled = scaler.fit_transform(X_additional)

# Combine TF-IDF reduced features with scaled additional features
X_combined = np.hstack((X_tfidf_reduced, X_additional_scaled))

# Define the target variable
y = combined_df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Build the ANN Model with a smaller structure
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8846 - loss: 0.3502 - val_accuracy: 0.9403 - val_loss: 0.1502
Epoch 2/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9423 - loss: 0.1485 - val_accuracy: 0.9428 - val_loss: 0.1434
Epoch 3/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9450 - loss: 0.1435 - val_accuracy: 0.9428 - val_loss: 0.1424
Epoch 4/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9453 - loss: 0.1428 - val_accuracy: 0.9433 - val_loss: 0.1418
Epoch 5/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9440 - loss: 0.1445 - val_accuracy: 0.9440 - val_loss: 0.1429
Epoch 6/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9450 - loss: 0.1425 - val_accuracy: 0.9437 - val_loss: 0.1412
Epoch 7/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9439 - loss: 0.1440 - val_accuracy: 0.9436 - val_loss: 0.1435
Epoch 8/50
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9451 - loss: 0.1412 - 